In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [46]:
%aiida
from apps.mcmasters.replicawork import ReplicaWorkchain
from aiida.work.process import WorkCalculation
from aiida_cp2k.calculations import Cp2kCalculation
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.folder import FolderData

from aiida.common.exceptions import NotExistent
from aiida.orm.data.base import Str

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

import matplotlib.pyplot as plt
from pprint import pprint

import ase.io
import tempfile

from tempfile import NamedTemporaryFile
from base64 import b64encode
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

def display_thumbnail(th):
    return '<img width="400px" src="data:image/png;base64,{}" title="">'.format(th)
def html_thumbnail(th):
    return ipw.HTML('<img width="400px" src="data:image/png;base64,{}" title="">'.format(th))

In [3]:
preproc_v = 0.01

In [104]:
def get_nebs():
    qb = QueryBuilder()
    qb.append(WorkCalculation,
              filters={
                  'attributes._process_label': {'==': 'NEBWorkchain'},
                  # 'attributes': {'!has_key': '_failed'}
                  'or': [
                      {'extras': {'!has_key': 'preproc_v'}},
                      {'extras.preproc_v': {'<': preproc_v}}
                  ]
              },
              tag='wc',
              project=['description', '*']
              #project=['description', 'ctime']
    )

    qb.append(Cp2kCalculation,
              output_of='wc',
              tag='cp2k',
              project='*'
    )

    qb.append(FolderData,
              output_of='cp2k',
              project='*',
              tag='retrieved folder')

    return qb.dict()

In [96]:
def preprocess(NEBlist):
    all_images = []
    for neb in NEBlist[0:1]:
        wc = neb['wc']['*']
        folder = neb['retrieved folder']['*']
        folder_list = folder.get_folder_list()
        folder_abs = folder.get_abs_path()+'/path/'
        images = []
        for i in range(len(folder_list)):
            fn = 'aiida-pos-Replica_nr_{num:02d}-1.xyz'.format(num=i+1)
            print fn
            print "{}/{}".format(i, len(folder_list))
            try:
                ase_struc = ase.io.read(folder_abs+fn, format='xyz')
                ase_struc.cell = str(wc.get_inputs_dict()['cell']).split()
                aiida_struc = StructureData(ase=ase_struc)
                images.append(aiida_struc)
                #wc.out('image {}'.format(i), aiida_struc)
            except IOError:
                break
        if len(images) > 0:
            all_images.append(images)
    return all_images

In [97]:
x = preprocess(get_nebs())

aiida-pos-Replica_nr_01-1.xyz
0/51
aiida-pos-Replica_nr_02-1.xyz
1/51
aiida-pos-Replica_nr_03-1.xyz
2/51
aiida-pos-Replica_nr_04-1.xyz
3/51
aiida-pos-Replica_nr_05-1.xyz
4/51
aiida-pos-Replica_nr_06-1.xyz
5/51
aiida-pos-Replica_nr_07-1.xyz
6/51
aiida-pos-Replica_nr_08-1.xyz
7/51
aiida-pos-Replica_nr_09-1.xyz
8/51
aiida-pos-Replica_nr_10-1.xyz
9/51
aiida-pos-Replica_nr_11-1.xyz
10/51
aiida-pos-Replica_nr_12-1.xyz
11/51
aiida-pos-Replica_nr_13-1.xyz
12/51
aiida-pos-Replica_nr_14-1.xyz
13/51
aiida-pos-Replica_nr_15-1.xyz
14/51
aiida-pos-Replica_nr_16-1.xyz
15/51
aiida-pos-Replica_nr_17-1.xyz
16/51
aiida-pos-Replica_nr_18-1.xyz
17/51
aiida-pos-Replica_nr_19-1.xyz
18/51
aiida-pos-Replica_nr_20-1.xyz
19/51
aiida-pos-Replica_nr_21-1.xyz
20/51
aiida-pos-Replica_nr_22-1.xyz
21/51
aiida-pos-Replica_nr_23-1.xyz
22/51
aiida-pos-Replica_nr_24-1.xyz
23/51
aiida-pos-Replica_nr_25-1.xyz
24/51
aiida-pos-Replica_nr_26-1.xyz
25/51
aiida-pos-Replica_nr_27-1.xyz
26/51
aiida-pos-Replica_nr_28-1.xyz
27/51
ai

In [99]:
x[0]

[<StructureData: uuid: d0c81153-1ff6-475f-a5ee-97ce7c4629f6 (unstored)>,
 <StructureData: uuid: d40d8e49-f142-4b60-a7ee-596e86dcb3b5 (unstored)>,
 <StructureData: uuid: 2dee0ea4-59ce-45d2-bf94-f02a2df0a143 (unstored)>,
 <StructureData: uuid: a4d74735-9b53-4601-8111-293587414a91 (unstored)>,
 <StructureData: uuid: b3f9787a-7c6f-43a1-a8ec-814dc25fb53b (unstored)>,
 <StructureData: uuid: 6b4da8d4-039b-414a-a716-f0d4869b1622 (unstored)>,
 <StructureData: uuid: a79831fd-edf6-40b6-b7e7-bc3e52d8fa69 (unstored)>,
 <StructureData: uuid: 7007d7f7-9b87-423b-ada0-5a104ebf0eee (unstored)>,
 <StructureData: uuid: 1f2fab3e-d19e-40b0-89e0-c9fb7cc11fca (unstored)>,
 <StructureData: uuid: d5a36b9c-f0f9-493f-943d-f53ca9c8d34e (unstored)>,
 <StructureData: uuid: f20dfe1f-53be-4b26-a481-097f8e705245 (unstored)>,
 <StructureData: uuid: 8ee85b37-fca9-4f81-bade-11fc518d5848 (unstored)>,
 <StructureData: uuid: 06c71577-295a-42a4-8636-ddd2684272c0 (unstored)>,
 <StructureData: uuid: 85439b78-c328-41ff-9adf-ebe3

In [112]:
wc = get_nebs()[0]['wc']['*']
jc = get_nebs()[0]['cp2k']['*']
wc.out.image1 = x[0][0]

In [107]:
get_nebs()[0]['cp2k']['*']

<Cp2kCalculation: uuid: 01cd5ac6-5400-476f-879c-46b7138b213e (pk: 53692)>

In [117]:
wc.out.image1 = x[0][0]
wc.out.image1.store()

AttributeError: Node 53672 does not have an output with link image1